In [1]:
#Importamos las bibliotecas que usaremos
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.linear_model import LinearRegression

data = pd.read_csv('data/data.csv',decimal=',',sep=';')
data.sample(3)



,patientcarequeue,appointmentfromtime,DayName,priordayholiday,nextdayholiday,day,hour,outofschedule,appointmentstate,age,...,speciality,patientaddress,gender,email,citypatient,buildingaddress,healthcoverage,buildingcity,medereentity,firsttime
57791,546.870,27/10/2021 16:00,Wednesday,0,0,27,16,True,1,9,...,Pediatría,C Y LEON 833,F,florcitaabalos@gmail.com,Mar del Plata,San Luis 2176,OAM,Mar del plata,36.555,0
20667,463.489,27/4/2021 10:00,Tuesday,0,0,27,10,True,1,12,...,Pediatría,BOUCHARD 4950,F,oscarudrizar@yahoo.com.ar,Mar del Plata,San Luis 2176,Sin OS,Mar del plata,23.589,0
82718,592.333,3/3/2022 13:30,Thursday,0,0,3,13,False,0,40,...,Gastroenterología adultos,W. MORRIS 9256,F,NaN,Mar del Plata,Av. Tejedor 264,NaN,Mar del plata,124.737,1


In [2]:

data['day'] = data['day'].fillna(0).astype(int)
data['hour'] = data['hour'].fillna(0).astype(int)
data['age'] = data['age'].fillna(0).astype(int)
data['firsttime'] = data['firsttime'].fillna(0).astype(int)
data['priordayholiday'] = data['priordayholiday'].fillna(0).astype(int)
data['nextdayholiday'] = data['nextdayholiday'].fillna(0).astype(int)

df_obj = data.select_dtypes(['object'])
data[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113139 entries, 0 to 113138
Data columns (total 21 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   patientcarequeue     113139 non-null  object
 1   appointmentfromtime  113139 non-null  object
 2   DayName              113139 non-null  object
 3   priordayholiday      113139 non-null  int32 
 4   nextdayholiday       113139 non-null  int32 
 5   day                  113139 non-null  int32 
 6   hour                 113139 non-null  int32 
 7   outofschedule        113139 non-null  bool  
 8   appointmentstate     113139 non-null  int64 
 9   age                  113139 non-null  int32 
 10  daysbeforereserved   113139 non-null  int64 
 11  speciality           113139 non-null  object
 12  patientaddress       87430 non-null   object
 13  gender               113139 non-null  object
 14  email                74500 non-null   object
 15  citypatient          113139 non-nu

In [3]:
import re

regex = re.compile(r'([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+')

def isValid(email):
    try:
        if re.fullmatch(regex, email):
          return 1 
        else:
          return 0
    except:
        return 0

data['emailvalid'] = data.apply(lambda x:isValid(x['email']),axis=1)

data.loc[data['healthcoverage'].isna(),'healthcoverage']= 'Sin OS' 
data.loc[data['healthcoverage'] != 'Sin OS','healthcoverage']= 'Con OS' 


In [4]:
data['endofmonth'] = 0
data.loc[data['day'] > 27,'endofmonth'] = 1

In [5]:
data.drop_duplicates(subset=['appointmentfromtime', 'medereentity'])


,patientcarequeue,appointmentfromtime,DayName,priordayholiday,nextdayholiday,day,hour,outofschedule,appointmentstate,age,...,gender,email,citypatient,buildingaddress,healthcoverage,buildingcity,medereentity,firsttime,emailvalid,endofmonth
0,422.104,4/1/2021 08:48,Monday,0,0,4,8,False,1,1,...,F,NaN,Mar del Plata,San Luis 2143,Con OS,Mar del plata,90.369,1,0,0
1,422.105,4/1/2021 08:49,Monday,0,0,4,8,False,0,1,...,F,NaN,Mar del Plata,San Luis 2143,Con OS,Mar del plata,90.369,0,0,0
2,422.109,4/1/2021 08:58,Monday,0,0,4,8,False,1,2,...,F,lula_adn@hotmail.com,Mar del Plata,San Luis 2176,Con OS,Mar del plata,78.015,1,1,0
3,417.480,4/1/2021 09:00,Monday,0,0,4,9,False,1,11,...,F,estefaniacoria@hotmail.com,Mar del Plata,San Luis 2176,Con OS,Mar del plata,14.948,1,1,0
4,418.219,4/1/2021 09:00,Monday,0,0,4,9,False,1,81,...,M,NaN,Mar del Plata,San Luis 2176,Con OS,Mar del plata,87.963,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113134,678.225,18/7/2022 18:20,Monday,0,0,18,18,True,1,69,...,F,NaN,Mar del Plata,San Luis 2176,Con OS,Mar del plata,107.746,0,0,0
113135,664.972,18/7/2022 18:40,Monday,0,0,18,18,False,1,45,...,F,crisbarresi@gmail.com,Mar del Plata,San Luis 2176,Con OS,Mar del plata,8.880,1,1,0
113136,673.656,18/7/2022 18:40,Monday,0,0,18,18,False,1,1,...,M,camilabernardo26@hotmail.com,Mar del Plata,San Luis 2143,Con OS,Mar del plata,107.869,0,1,0
113137,676.097,18/7/2022 18:40,Monday,0,0,18,18,False,1,4,...,M,veronica.luduena@gmail.com,Mar del Plata,Carlos Alvear 2818,Con OS,Mar del plata,13.383,0,1,0


In [6]:
data = data.drop(['medereentity','patientcarequeue','patientaddress', 'email','buildingaddress'], axis=1)

In [7]:
data.isna().sum()

appointmentfromtime    0
DayName                0
priordayholiday        0
nextdayholiday         0
day                    0
hour                   0
outofschedule          0
appointmentstate       0
age                    0
daysbeforereserved     0
speciality             0
gender                 0
citypatient            0
healthcoverage         0
buildingcity           0
firsttime              0
emailvalid             0
endofmonth             0
dtype: int64

In [8]:
data.dropna(inplace=True)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113139 entries, 0 to 113138
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   appointmentfromtime  113139 non-null  object
 1   DayName              113139 non-null  object
 2   priordayholiday      113139 non-null  int32 
 3   nextdayholiday       113139 non-null  int32 
 4   day                  113139 non-null  int32 
 5   hour                 113139 non-null  int32 
 6   outofschedule        113139 non-null  bool  
 7   appointmentstate     113139 non-null  int64 
 8   age                  113139 non-null  int32 
 9   daysbeforereserved   113139 non-null  int64 
 10  speciality           113139 non-null  object
 11  gender               113139 non-null  object
 12  citypatient          113139 non-null  object
 13  healthcoverage       113139 non-null  object
 14  buildingcity         113139 non-null  object
 15  firsttime            113139 non-nu

In [10]:
data.to_csv("data/dataset_turnos.csv")